# Text Summarization Project

As outlined in this walkthrough: https://stackabuse.com/text-summarization-with-nltk-in-python/

### Convert Paragraphs to Sentences

In [150]:
someText = open('gettysbergAddress.txt','r',encoding="UTF-8")

In [151]:
someText = someText.read()

In [152]:
print(someText)

Fourscore and seven years ago our fathers brought forth on this continent a new nation, conceived in liberty and dedicated to the proposition that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battlefield of that war. We have come to dedicate a portion of that field as a final resting-place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this. But, in a larger sense, we cannot dedicate — we cannot consecrate — we cannot hallow — this ground. The brave men, living and dead, who struggled here have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so no

In [153]:
# We have to split on "—" and "."
someText2 = ""
for char in someText:
    if char == '—':
        someText2 += "." # replace "—" with "." so we can call split once
    else:
        someText2 += char
        
sentences = someText2.split(".")

In [154]:
sentences = someText2.split(".")

In [155]:
# Return Space
sentences2 = []
for sentence in sentences:
    sentences2.append(sentence.rstrip())

In [156]:
sentences2

['Fourscore and seven years ago our fathers brought forth on this continent a new nation, conceived in liberty and dedicated to the proposition that all men are created equal',
 '\n\nNow we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure',
 ' We are met on a great battlefield of that war',
 ' We have come to dedicate a portion of that field as a final resting-place for those who here gave their lives that that nation might live',
 ' It is altogether fitting and proper that we should do this',
 ' But, in a larger sense, we cannot dedicate',
 ' we cannot consecrate',
 ' we cannot hallow',
 ' this ground',
 ' The brave men, living and dead, who struggled here have consecrated it, far above our poor power to add or detract',
 ' The world will little note, nor long remember what we say here, but it can never forget what they did here',
 ' It is for us the living, rather, to be dedicated here to the unfinished work w

### Text Preprocessing

In [157]:
# Whitespace
sentences3 = []
for sentence in sentences2:
    sentences3.append(sentence.strip())

In [158]:
# Empty strings
sentences4 = []
for sentence in sentences3:
    if len(sentence) >0:
        sentences4.append(sentence)

In [159]:
sentencesLater = [sentence.capitalize() for sentence in sentences4] # sentencesLater will be used in a later step
sentences = sentences4


Punctuation Removal

In [160]:
import string
nopunc = string.punctuation

In [161]:
# Remove punctuation
sentencesA = []
for sentence in sentences:
    temp = ""
    for char in sentence:
        if char not in nopunc:
            if char != "-": # toggle
                temp += char
    sentencesA.append(temp)

In [162]:
# Lowercase
sentencesB = []
for sentence in sentencesA:
    sentencesB.append(sentence.lower())

In [163]:
sentences = sentencesB

### Tokenizing the Sentences

Creating a list of meaningful words

In [164]:
words = []

In [165]:
from nltk.corpus import stopwords

In [166]:
for sentence in sentences:
    temp = sentence.split()
    for word in temp:
        if word not in stopwords.words('english'):
            words.append(word)

In [167]:
words

['fourscore',
 'seven',
 'years',
 'ago',
 'fathers',
 'brought',
 'forth',
 'continent',
 'new',
 'nation',
 'conceived',
 'liberty',
 'dedicated',
 'proposition',
 'men',
 'created',
 'equal',
 'engaged',
 'great',
 'civil',
 'war',
 'testing',
 'whether',
 'nation',
 'nation',
 'conceived',
 'dedicated',
 'long',
 'endure',
 'met',
 'great',
 'battlefield',
 'war',
 'come',
 'dedicate',
 'portion',
 'field',
 'final',
 'restingplace',
 'gave',
 'lives',
 'nation',
 'might',
 'live',
 'altogether',
 'fitting',
 'proper',
 'larger',
 'sense',
 'cannot',
 'dedicate',
 'cannot',
 'consecrate',
 'cannot',
 'hallow',
 'ground',
 'brave',
 'men',
 'living',
 'dead',
 'struggled',
 'consecrated',
 'far',
 'poor',
 'power',
 'add',
 'detract',
 'world',
 'little',
 'note',
 'long',
 'remember',
 'say',
 'never',
 'forget',
 'us',
 'living',
 'rather',
 'dedicated',
 'unfinished',
 'work',
 'fought',
 'thus',
 'far',
 'nobly',
 'advanced',
 'rather',
 'us',
 'dedicated',
 'great',
 'task',
 '

### Find Weighted Frequency of Occurrence

Finding the weight of words relative to the complete text

In [168]:
import pandas as pd

In [169]:
freqDict = {}
for word in words:
    if word not in freqDict:
        freqDict[word] = 0
    else:
        freqDict[word] += 1
        

In [170]:
setWords = set(words)
setWordsList = list(setWords)

In [171]:
# Word Frequency DF
df = pd.DataFrame(data=[[i, freqDict[i]] for i in setWordsList ], columns=["Word","Freq"])

In [172]:
df.head()

,Word,Freq
0,living,1
1,government,0
2,rather,1
3,birth,0
4,testing,0


In [173]:
df.sort_values(by="Freq",ascending=False, inplace=True)

In [174]:
df

,Word,Freq
26,nation,4
63,dedicated,3
47,cannot,2
58,great,2
37,dead,2
...,...,...
33,world,0
32,fourscore,0
31,fathers,0
30,power,0


In [175]:
df['wFreq'] = df['Freq'].apply(lambda x: x/5)

In [176]:
df.head()

,Word,Freq,wFreq
26,nation,4,0.8
63,dedicated,3,0.6
47,cannot,2,0.4
58,great,2,0.4
37,dead,2,0.4


### Replace Words by Weighted Frequency in Original Sentences

Giving weight to sentences based on their word composition relative to the corpus

In [177]:
# sum of weighted frequencies
swf = []

In [178]:
swfDict = {}
wordArray = df['Word']
wFreq = df['wFreq']

In [179]:
for i in range(len(wordArray)):
    swfDict[wordArray[i]] = wFreq[i]

In [180]:
swfList = [ ]

In [181]:
for sentence in sentences:
    temp = sentence.split()
    score = 0
    for i in temp:
        if i in swfDict:
            score += swfDict[i]
    swfList.append(score)
            
            

In [182]:
# Scores Sentences DF (using sentencesLater)
df2 = pd.DataFrame(data=[[sentencesLater[i], swfList[i]] for i in range(len(swfList)) ], columns=["Sentence","Score"])

In [183]:
df2.sort_values(by="Score",ascending=False, inplace=True)

In [184]:
df2.reset_index(drop=True, inplace=True)

In [185]:
df2

,Sentence,Score
0,"Now we are engaged in a great civil war, testi...",3.2
1,That this nation shall have a new birth of fre...,3.0
2,It is rather for us to be here dedicated to th...,2.0
3,Fourscore and seven years ago our fathers brou...,2.0
4,"It is for us the living, rather, to be dedicat...",1.6
5,We have come to dedicate a portion of that fie...,1.2
6,"The brave men, living and dead, who struggled ...",1.0
7,That from these honored dead we take increased...,1.0
8,That we here highly resolve that these dead sh...,0.8
9,We are met on a great battlefield of that war,0.6


## numLines

A function that returns a summarized speech (given some number of sentences)

In [186]:
def numLines(num):
    temp = ""
    for i in df2['Sentence'].head(num):
        temp += i
        temp += "."
        temp += " "
    
    print(temp)

In [191]:
# use numLines(n) to summarize the text to n-many sentences
numLines(5)

Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. That this nation shall have a new birth of freedom and that government of the people, by the people, for the people, shall not perish from the earth. It is rather for us to be here dedicated to the great task remaining before us. Fourscore and seven years ago our fathers brought forth on this continent a new nation, conceived in liberty and dedicated to the proposition that all men are created equal. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced. 


### Conclusion

Are goal was to summarize Abraham Lincoln's Gettysburg Address. To do this we pre-processed, tolkenized words of sentences to give sentences weight, and ranked sentences. Our function "numLines(n)" summarizes the text by returning heaviest 'n' sentences from the rankings. With this Project one can summarize bodies of text. 